In [ ]:
# imports required to run the code.
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import sklearn
import autograd.numpy as np_
import numpy as np
from autograd import grad

In [ ]:
# preparing dataset.

dfo = pd.read_csv('healthcare-dataset-stroke-data.csv')
dfo = dfo.fillna(0)

# ---------------------------------------- #

dfo.head()

In [ ]:
# preparing dataset.

dfo = pd.read_csv('healthcare-dataset-stroke-data.csv')
dfo = dfo.fillna(0)

dfo = dfo.sort_values(by=['bmi'], ascending=True)
dfo = dfo.reset_index(drop = True)

# ---------------------------------------- #

df = pd.get_dummies(dfo) # converte dados não numéricos.
df = df.drop(columns=['gender_Other', 'gender_Male']) # remoção de dados redundantes.
df.rename(columns={"gender_Female": "gender"}, inplace = True)

# ---------------------------------------- #
df.head()

In [ ]:
# ----------------- #

filtery = (df['stroke'] == 0)
dfo1 = df[filtery]

dfo1 = dfo1.iloc[:300]

# ----------------- #

filtery = (df['stroke'] == 1)
dfo2 = df[filtery]

dfo = dfo1.append(dfo2)

dfo

In [ ]:
dfa = dfo.drop(columns=['stroke','id','ever_married_No','Residence_type_Rural','smoking_status_formerly smoked']).astype(float)

X_train, X_test, y_train, y_test = train_test_split(dfa, dfo['stroke'], train_size=0.5)
y_train = y_train.replace(to_replace=0,value=-1)
y_test = y_test.replace(to_replace=0,value=-1)
y_train.value_counts()

In [ ]:
X_train,X_test,y_train,y_test = X_train.to_numpy(),X_test.to_numpy(),y_train.to_numpy(),y_test.to_numpy()

In [ ]:
import autograd.numpy as np_
import numpy as np
from autograd import grad

# ---------------------------------------- #

def predict( w, b, pontos ):
    est = w.T @ pontos + b
    return est


def accuracy(y_test, y_est):
    return np.mean(np.sign(y_test)==np.sign(y_est))

def loss( parametros ):
    w, b, pontos, val = parametros
    est = w.T @ pontos + b
    mse = np_.mean( (est - val)**2)
    return mse

# ---------------------------------------- #

In [ ]:
results = []
for _ in range(10):
    g = grad(loss)

    pontos = X_train.T
    alvos = (y_train).astype(float)

    w = np.random.randn( pontos.shape[0],1)
    b = 0.0
    alpha = 0.000001

    for n in range(1000):
        grad_ = g( (w, b, pontos, alvos) )
        w -= alpha*grad_[0]
        b -= alpha*grad_[1]

    y_pred = (predict( w, b, X_test.T )).reshape((y_test.shape))

    results.append(accuracy(y_test, y_pred))

In [ ]:
print(np.array(results).mean())
print(np.array(results).std())

In [ ]:
g = grad(loss)

pontos = X_train.T
alvos = (y_train).astype(float)

w = np.random.randn( pontos.shape[0],1)
b = 0.0
alpha = 0.000001
print(w)
for n in range(1000):
    grad_ = g( (w, b, pontos, alvos) )
    w -= alpha*grad_[0]
    b -= alpha*grad_[1]

y_pred = (predict( w, b, X_test.T )).reshape((y_test.shape))

In [ ]:
from collections import OrderedDict

columns = list(dfa.columns)
res = {columns[i]: float(w[i]) for i in range(len(w))}
res_sorted = sorted(res.items(), key=lambda t: t[1], reverse=True)

[('smoking_status_never smoked', 1.9662127728927499),
 ('work_type_Govt_job', 1.5288039518764849),
 ('heart_disease', 1.3447350243293723),
 ('work_type_Private', 0.7072463824185442),
 ('ever_married_Yes', 0.4894299210636069),
 ('hypertension', 0.26751445224545006),
 ('work_type_children', 0.11226938267801652),
 ('bmi', 0.028330032893819176),
 ('gender', 0.0008012721255334959),
 ('avg_glucose_level', -0.0048443433053298015),
 ('age', -0.007321216850998305),
 ('smoking_status_smokes', -0.6108328187336413),
 ('Residence_type_Urban', -0.6731651799177507),
 ('smoking_status_Unknown', -0.7852556759763394),
 ('work_type_Never_worked', -1.1298236983714267),
 ('work_type_Self-employed', -1.7866693388941148)]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

print(dfa)
df_features = pd.DataFrame(X_test)
print(df_features)
df_rotulo = pd.DataFrame(y_test)
tree = DecisionTreeClassifier(criterion='entropy')

# Agora, vamos usar o método .fit() para ajustar os parâmetros da árvore:
tree.fit(df_features, df_rotulo)

# Podemos visualizar a árvore de decisão em uma figura!
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure( figsize=(20,20) )
a = plot_tree(tree, feature_names=df_features.columns, fontsize=15, 
              node_ids=False, impurity=False, filled=True)

#a